In [1]:
import random
import copy

In [2]:
# Initialize a random population
def initialize_population(pop_size):
    population = []
    for _ in range(pop_size):
        individual = random.sample(range(BOARD_SIZE), BOARD_SIZE)
        population.append(individual)
    return population

In [3]:
# Calculate the penalty for a given individual
def calculate_penalty(individual):
    penalty = 0
    for i in range(BOARD_SIZE):
        for j in range(i + 1, BOARD_SIZE):
            if individual[i] == individual[j] or abs(individual[i] - individual[j]) == j - i:
                penalty += 1
    return penalty

In [4]:
# Calculate the fitness of an individual (inverse of penalty)
def calculate_fitness(individual):
    penalty = calculate_penalty(individual)
    if penalty == 0:
        return float('inf')  # Ideal fitness when no penalties
    return 1 / (1 + penalty)

In [5]:
# Tournament Selection: Select the best individual from a random set
def tournament_selection(population, tournament_size=5):
    selected = random.sample(population, tournament_size)
    selected.sort(key=lambda x: calculate_fitness(x), reverse=True)
    return selected[0], selected[1]  # Return the top two individuals


In [6]:
# Single-Point Crossover
def single_point_crossover(parent1, parent2):
    crossover_point = random.randint(1, BOARD_SIZE - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

In [7]:
# Cycle Crossover (CX)
def cycle_crossover(parent1, parent2):
    child1 = [-1] * BOARD_SIZE
    child2 = [-1] * BOARD_SIZE
    cycle_start = 0

    while -1 in child1:
        if child1[cycle_start] == -1:
            cycle_value = parent1[cycle_start]
            while parent2[cycle_start] != cycle_value:
                cycle_start = parent1.index(parent2[cycle_start])
                cycle_value = parent1[cycle_start]
            child1[cycle_start] = parent1[cycle_start]
            child2[cycle_start] = parent2[cycle_start]
    return child1, child2


In [8]:
# Order-Based Crossover (OX)
def order_based_crossover(parent1, parent2):
    start, end = sorted(random.sample(range(BOARD_SIZE), 2))
    child1 = [-1] * BOARD_SIZE
    child2 = [-1] * BOARD_SIZE

    # Copy the slice from parent1 to the child
    child1[start:end+1] = parent1[start:end+1]
    child2[start:end+1] = parent2[start:end+1]

    # Fill the remaining positions with the genes from parent2 and parent1 respectively
    p1_fill = [gene for gene in parent2 if gene not in child1]
    p2_fill = [gene for gene in parent1 if gene not in child2]
    
    for i in range(BOARD_SIZE):
        if child1[i] == -1:
            child1[i] = p1_fill.pop(0)
        if child2[i] == -1:
            child2[i] = p2_fill.pop(0)

    return child1, child2

In [9]:
# Swap Mutation
def swap_mutation(individual):
    i, j = random.sample(range(BOARD_SIZE), 2)
    individual[i], individual[j] = individual[j], individual[i]
    return individual


In [ ]:
# Swap Mutation
def swap_mutation2(individual):
    i, j = random.sample(range(BOARD_SIZE), 2)
    individual[i], individual[j] = individual[j], individual[i]
    m, n = random.sample(range(BOARD_SIZE), 2)
    individual[m], individual[n] = individual[n], individual[m]
    return individual

In [11]:
def inversion_mutation(individual):
    # Step 1: Choose two random points to define the subset range
    idx1, idx2 = sorted(random.sample(range(len(individual)), 2))
    
    # Step 2: Reverse the subset within the selected range
    individual[idx1:idx2+1] = individual[idx1:idx2+1][::-1]
    
    return individual

In [12]:
# Replace less fit individuals
def replace_population(population, new_individuals):
    population.sort(key=lambda x: calculate_fitness(x), reverse=True)
    for i in range(len(new_individuals)):
        if calculate_fitness(new_individuals[i]) > calculate_fitness(population[i]):
            population[i] = new_individuals[i]
    return population

In [33]:
# Main Genetic Algorithm Function
def genetic_algorithm(flag,population,pop_size=100, generations=1000, crossover_method=None):

    for generation in range(generations):
        if generation%100 ==0:
            print( generation)
        new_population = []

        # Perform Selection, Crossover, and Mutation
        while len(new_population) < pop_size:
            parent1, parent2 = tournament_selection(population)
            
            # Use the provided crossover method
            child1, child2 = crossover_method(parent1, parent2)
            
            if flag==1:
                # Apply mutation
                child1 = swap_mutation(child1)
                child2 = swap_mutation(child2)

        # Replace old population with new individuals
        population = replace_population(population, new_population)

        # Check for a valid solution
        best_individual = max(population, key=lambda x: calculate_fitness(x))
        if calculate_fitness(best_individual) == float('inf'):
            print(f"Solution found in generation {generation}")
            print(f"Best individual: {best_individual}")
            break
    else:
        print("Solution not found within the maximum number of generations.")





In [31]:
# Run the genetic algorithm with different crossover methods
def run_experiment(pop_size=100):
    pop1 = initialize_population(pop_size)
        
    # Experiment 3: Using Order-Based Crossover (OX)
    print("\nOX, Swap:")
    genetic_algorithm(1,pop1,pop_size=100, generations=1000, crossover_method=order_based_crossover)

    
    

In [32]:
# Define the size of the board
BOARD_SIZE = 30
# Run the three experiments
run_experiment()



OX, Swap:
0
100
200
300
Solution found in generation 369
Best individual: [13, 19, 22, 14, 27, 2, 5, 12, 29, 24, 26, 23, 6, 3, 0, 11, 16, 10, 21, 4, 9, 1, 28, 7, 25, 20, 18, 8, 15, 17]

OX, inverse
0
100
200
300
400
Solution found in generation 495
Best individual: [7, 13, 21, 25, 12, 3, 27, 10, 1, 9, 14, 28, 23, 0, 16, 11, 5, 26, 24, 18, 2, 4, 19, 8, 29, 15, 6, 22, 20, 17]
